# Question 3

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels

In [2]:
df = DataFrame(CSV.File("data/ps1_ex3.csv"));

# I need to get the 0 share in each market 
gdf = groupby(df, :market)
gdf = combine(gdf, :Shares => sum)
gdf[!,:shares_0] = 1 .- gdf[!,:Shares_sum];
df = leftjoin(df, gdf, on=:market);

# Will need ln(s_jt / s_0t) and ln(s_jt)
df[!,:log_sj_s0] = log.(df[!,:Shares]./df[!,:shares_0]);
df[!,:log_sj] = log.(df[!,:Shares]);

# Part 1: Estimating $(\alpha, \beta)$

We have the moment condition $E[\xi_{jt} | z_{jt}] = 0$, hence we can estimate the following with TSLS: 

$$ ln\left(\frac{s_{jt}}{s_{0t}}\right) = - \alpha p_{jt} + x_{jt}' \beta + \xi_{jt}$$

In [3]:
estimates = reg(df, @formula(log_sj_s0 ~ x + (Prices ~ z)))
show(estimates)
α = -coef(estimates)[3];
β = coef(estimates)[2];
df[!,:ξ] = (df[!,:log_sj_s0] - predict(estimates,df)) .+ coef(estimates)[1];

print("α: $α", "\nβ: $β")

                                IV Model                                
Number of obs:                 6000  Degrees of freedom:               2
R2:                           0.002  R2 Adjusted:                  0.001
F-Stat:                     787.122  p-value:                      0.000
F-Stat (First Stage):       2011.28  p-value (First Stage):        0.000
log_sj_s0   |  Estimate  Std.Error  t value Pr(>|t|) Lower 95% Upper 95%
------------------------------------------------------------------------
x           |  0.304672 0.00833715   36.544    0.000  0.288329  0.321016
Prices      | -0.467477  0.0618097 -7.56317    0.000 -0.588646 -0.346308
(Intercept) |  0.728887   0.194486  3.74775    0.000  0.347624   1.11015
α: 0.4674770969358192
β: 0.3046723858039903

# Part 2: Finding price elasticities

$$\eta_{jkt} = \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = \begin{cases}
        - \alpha p_{jt} (1-s_{jt}) & j=k \\
        \alpha p_{kt} s_{jt} & j\neq k
    \end{cases}$$

In [4]:
# reshape dataframe of prices and shares
share_array = Array(unstack(df, :market, :Product, :Shares)[!,2:7]);
price_array = Array(unstack(df, :market, :Product, :Prices)[!,2:7]);

In [114]:
# for each market, we should get a JxJ matrix 
T = size(share_array)[1]
J = size(share_array)[2]

η = zeros(T, J^2)

for t in 1:T
    iter = 1
    for j in 1:J
        for k in 1:J
            if j == k 
                η[t, iter] = -1 * α * price_array[t,j] * (1-share_array[t,k])
            else 
                η[t, iter] = α * price_array[t,k] * share_array[t,j]
            end
        iter += 1
        end
    end
end

# average across each market
η̄ = reshape(mean(η, dims=1), 6, 6)

6×6 Matrix{Float64}:
 -1.24962    0.317914   0.141565   0.139219   0.13808    0.142108
  0.317425  -1.25124    0.141836   0.13975    0.138349   0.142341
  0.286686   0.287691  -1.28915    0.126127   0.124954   0.128527
  0.287309   0.288337   0.128271  -1.29387    0.125171   0.128882
  0.28665    0.287474   0.127996   0.125908  -1.29143    0.128494
  0.287242   0.288019   0.128341   0.126219   0.125208  -1.2908

# Part 3: Recover marginal costs
Assuming that each product-market is a different firm, we can obtain the following from the FOC from the firm optimization:
$$ s_{jt}(p) + (p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{kt}} = 0 $$
which implies:
$$ s_{jt}(p) \frac{1}{\frac{\partial s_{jt}}{\partial p_{kt}}} + p_{jt} = mc_{jt} $$

We can equivalently write this as:
$$ mc_{jt} = p_{jt} - \frac{1}{\alpha}\frac{1}{(1-s_{jt})}$$
which makes it clear that average marginal costs can be related to prices and shares directly from the data given an estimate of $\alpha$.

In [4]:
# own partial
#df[!,:own_partial] = α .* df[!,:Shares] .* (1 .- df[!,:Shares]);
#df[!,:mc] = df[!,:Shares] ./ df[!,:own_partial] .+ df[!,:Prices];

df[!,:invshares] = 1 ./ (1 .- df[!,:Shares]);
df[!,:mc] = df[!,:Prices] - (1/α) .* df[!,:invshares];

means = groupby(df, :Product)
means = combine(means, [:mc,:Prices,:invshares, :Shares] .=> mean)

Row,Product,mc_mean,Prices_mean,invshares_mean,Shares_mean
,Int64,Float64,Float64,Float64,Float64
1,1,0.667126,3.35995,1.25883,0.202451
2,2,0.671897,3.36753,1.26015,0.203076
3,3,0.678684,3.03306,1.10061,0.0903493
4,4,0.688906,3.03977,1.09898,0.0889407
5,5,0.682632,3.03103,1.09782,0.0881716
6,6,0.682689,3.03815,1.10112,0.0906875


# Part 4: Product 1 exits the market
I think the following works:

- The regression predicts $ln(S_j/S_0)$, where $S_0 = 1 - \sum_k S_k$. Note $\sum_j S_j / S_0 = \frac{1-S_0}{S_0}$:
$$ \exp\left(\ln\left(\frac{S_j}{S_0}\right)\right) = \frac{S_j}{S_0} \\  \frac{\frac{S_j}{S_0}}{ \sum_j \frac{S_j}{S_0}} =  \frac{S_j}{1-S_0} = \frac{S_j}{\sum_k S_k} $$

MM: To simulate $j = 1$ leaving the market, we solve for the market prices that minimize the firms' first-order conditions as the objective, subject to demand (expressed as shares) as estimated above. We can use the fact that $\frac{\partial s_{k}}{\partial p_{k}} = -\alpha s_k (1-s_k)$. We can then plug the prices into the share equation to get simulated market shares. That is:
$$ \min_{p} || FOC_k ||  = || s(p) + \alpha (p - c)s(p)(1 - s(p)) || $$
$$ \text{s.t. } s_{j} = \frac{\exp(-\alpha p_{j} + x_{j}'\beta + \xi_{j})}{1 + \sum_{k = 2}^5 \exp(-\alpha p_{k} + x_{k}'\beta + \xi_{k})}$$ 
We can solve this with a simple fixed point algorithm, starting with a candidate vector of market prices, $p_k^{(i)}$, then calculating the implied price from the FOC by $p_k^{(i+1)} = c - \frac{1}{\alpha(1-s(p_k^{(i)}))}$, until some tolerance for $p_k^{(i+1)} - p_k^{(i)}$.

In [68]:
function implied_shares(J,x,ξ,α,β,p) # calculate market demand given parameters, prices, and products
    # numerator
    u = zeros(J);
    @. u = exp(-α * p + x * β + ξ);
    
    # denominator
    u_sum = sum(u) + 1;
    
    # calculate shares
    s = u ./ u_sum;
    
    return s
end

function implied_p(x,ξ,α,β,c,p)
    J = size(p)
    
    s = implied_shares(J,x,ξ,α,β,p)
    # p = c + 1/α .* (1 ./ (1 .- s))
    p = c + 1/α .* s .* (p - c)   
    
    return p
end

function foc(J,α,s,c,p)
    # calculate firm FOCs
    foc = zeros(J)
    @. foc = s + (α * (p - c) * s * (1 - s)); 
    
    return foc
end

function objective(x,ξ,α,β,c,p) # minimize total error in firm pricing FOCs in a market
    # number of parameters
    J = size(p)
    
    s = implied_shares(J,x,ξ,α,β,p)
    firmfoc = foc(J,α,s,c,p)
    
    # objective
    F = sum(firmfoc.^2)
    
    return F
end

objective (generic function with 1 method)

In [96]:
# Option 1: Solver (I don't think this works right, prices can be ~1% off)
simulation = df[!,[:market, :Product, :x, :ξ, :mc, :Prices, :Shares]];
simulation = simulation[(df.Product .!= 1), :];
nmkts = 1000

# Solve for prices, looping through each market
prices1 = [];

focs1 = [];
shares1 = [];

n_params = size(simulation[(simulation.market .== 1), :])[1]; 

count = 0
@time for mkt in 1:nmkts
    sim_mkt = simulation[(simulation.market .== mkt), :];

    # start somewhere close to true prices
    lower =  Array(ones(n_params) .* 1.0)
    # params0 = Array(ones(n_params) .* 3.0)
    params0 = Array(sim_mkt[!,:Prices])
    upper =  Array(ones(n_params) .* 6.0)  

    x = Array(sim_mkt[!,:x]);
    ξ = Array(sim_mkt[!,:ξ]);
    c = Array(sim_mkt[!,:mc]);

    optimum = optimize(p -> objective(x,ξ,α,β,c,p), lower, upper, params0, Fminbox(LBFGS()))
    count += (Optim.converged(optimum) == true)
    p_sim = optimum.minimizer

    push!(prices1,p_sim)

    o = objective(x,ξ,α,β,c,p_sim)
    s = implied_shares(n_params,x,ξ,α,β,p_sim)

    push!(focs1,o)
    push!(shares1,s)
end
print("successes: ", count, "/", nmkts)

  0.658797 seconds (4.82 M allocations: 346.301 MiB, 11.07% gc time, 14.94% compilation time)
successes: 1000/1000

In [67]:
# Option 2: Fixed point (this is spot on accurate)
simulation = df[!,[:market, :Product, :x, :ξ, :mc, :Prices, :Shares]];
# simulation = simulation[(df.Product .!= 1), :];
nmkts = 1000
tol = 1e-16

# Solve for prices, looping through each market
prices2 = [];

focs2 = [];
shares2 = [];
fail = []

n_params = size(simulation[(simulation.market .== 1), :])[1]; 

count = 0
@time for mkt in 1:1000
    sim_mkt = simulation[(simulation.market .== mkt), :];
    
    # start somewhere close to true prices
    # p₀ = Array(ones(n_params) .* 1.0)
    p₀ = Array(sim_mkt[!,:Prices]) 
    
    x = Array(sim_mkt[!,:x]);
    ξ = Array(sim_mkt[!,:ξ]);
    c = Array(sim_mkt[!,:mc]);
    
    pᵢ = p₀
    ε = 1
    i = 0
    while (ε > tol) && (i < 1000)   
        i += 1
        pᵢ₊₁ = implied_p(x,ξ,α,β,c,pᵢ)
        ε = maximum(abs.(pᵢ₊₁ - pᵢ))
        pᵢ = max.(pᵢ₊₁,zeros(n_params))
    end

    #println("mkt: $mkt", " ε: $ε", " i: $i")
    #println("p: $pᵢ")  
    
    append!(prices2,pᵢ)
    
    o = objective(x,ξ,α,β,c,pᵢ)
    s = implied_shares(n_params,x,ξ,α,β,pᵢ)
    
    push!(focs2,o)
    push!(shares2,s)
    push!(fail,(i == 1000))
end

print("Maxed out iterations: ", sum(fail), "\nMax L2-FOC distance: ", maximum(focs2))

  1.180757 seconds (5.82 M allocations: 422.507 MiB, 7.55% gc time, 21.67% compilation time)
Maxed out iterations: 403
Max L2-FOC distance: 0.7685875525307537

In [59]:
s = simulation[(simulation.market .<= nmkts), :Shares]
p = simulation[(simulation.market .<= nmkts), :Prices]

println("Old shares and means: ", means[!,["Prices_mean", "Shares_mean"]])
#println("New prices1: ", mean(prices1, dims = 1))
#println("New shares1: ", mean(shares1, dims = 1))
println("New prices2: ", mean(prices2, dims = 1))
println("New shares2: ", mean(shares2, dims = 1))

Old shares and means: 6×2 DataFrame
 Row │ Prices_mean  Shares_mean 
     │ Float64      Float64     
─────┼──────────────────────────
   1 │     3.35995    0.202451
   2 │     3.36753    0.203076
   3 │     3.03306    0.0903493
   4 │     3.03977    0.0889407
   5 │     3.03103    0.0881716
   6 │     3.03815    0.0906875
8.881784197001252e-16
